In [ ]:
import React from 'react';
import { render, screen } from '@testing-library/react';
import { vi } from 'vitest';
import ZnaiSelect from '../ZnaiSelect';

In [ ]:
vi.mock('react-select', () => ({
  default: ({ options, value, onChange, ...props }: any) => (
    <select
      data-testid="react-select"
      value={value?.value || ''}
      onChange={(e) => {
        const selectedOption = options.find((opt: any) => opt.value === e.target.value);
        onChange?.(selectedOption);
      }}
      {...props}
    >
      {options?.map((option: any) => (
        <option key={option.value} value={option.value}>
          {option.label}
        </option>
      ))}
    </select>
  ),
}));

In [ ]:
const sampleOptions = [
  { value: 'chocolate', label: 'Chocolate' },
  { value: 'strawberry', label: 'Strawberry' },
  { value: 'vanilla', label: 'Vanilla' }
];

const defaultProps = {
  options: sampleOptions,
  selectedOption: null,
  handleChange: vi.fn()
};

In [ ]:
describe('ZnaiSelect - Basic Rendering', () => {
  test('renders without crashing', () => {
    render(<ZnaiSelect {...defaultProps} />);
    expect(screen.getByTestId('react-select')).toBeInTheDocument();
  });

  test('renders Select component', () => {
    render(<ZnaiSelect {...defaultProps} />);
    const selectElement = screen.getByTestId('react-select');
    expect(selectElement).toBeInTheDocument();
  });
});

In [ ]:
describe('ZnaiSelect - Props Passing', () => {
  test('passes options correctly to Select component', () => {
    render(<ZnaiSelect {...defaultProps} />);
    const selectElement = screen.getByTestId('react-select');

    sampleOptions.forEach(option => {
      expect(screen.getByText(option.label)).toBeInTheDocument();
    });
  });

  test('passes selectedOption correctly', () => {
    const selectedOption = sampleOptions[1];
    render(<ZnaiSelect {...defaultProps} selectedOption={selectedOption} />);

    const selectElement = screen.getByTestId('react-select') as HTMLSelectElement;
    expect(selectElement.value).toBe(selectedOption.value);
  });
});

In [ ]:
describe('ZnaiSelect - Interaction', () => {
  test('calls handleChange when selection changes', () => {
    const mockHandleChange = vi.fn();
    const consoleSpy = vi.spyOn(console, 'log').mockImplementation(() => {});

    render(<ZnaiSelect {...defaultProps} handleChange={mockHandleChange} />);

    const selectElement = screen.getByTestId('react-select');
    selectElement.dispatchEvent(new Event('change', { bubbles: true }));

    expect(mockHandleChange).toHaveBeenCalled();

    consoleSpy.mockRestore();
  });

  test('logs correct parameters when handleChange is called', () => {
    const consoleSpy = vi.spyOn(console, 'log').mockImplementation(() => {});

    render(<ZnaiSelect {...defaultProps} />);

    const selectElement = screen.getByTestId('react-select') as HTMLSelectElement;
    selectElement.value = 'chocolate';
    selectElement.dispatchEvent(new Event('change', { bubbles: true }));

    expect(consoleSpy).toHaveBeenCalledWith('Selected option:', expect.objectContaining({
      value: 'chocolate',
      label: 'Chocolate'
    }));

    consoleSpy.mockRestore();
  });
});

In [ ]:
describe('ZnaiSelect - Edge Cases', () => {
  test('handles empty options array', () => {
    render(<ZnaiSelect {...defaultProps} options={[]} />);
    const selectElement = screen.getByTestId('react-select');
    expect(selectElement).toBeInTheDocument();
    expect(selectElement.children.length).toBe(0);
  });

  test('handles selectedOption as first option (index 0)', () => {
    const selectedOption = sampleOptions[0];
    render(<ZnaiSelect {...defaultProps} selectedOption={selectedOption} />);

    const selectElement = screen.getByTestId('react-select') as HTMLSelectElement;
    expect(selectElement.value).toBe(selectedOption.value);
  });
});